# On arbor synapse distributions
* Now that we have synapses annotated for neurons, one of the analyses we want to do is look at the spatial distribution of synapses on our neuron of interest. 
* To do this, we need a few things:
1. A way to measure distances along arbors and calculate branch points.
2. A way to associate synapses with our neuron.

### Here we will use meshwork to calculate synapse density along our arbors 
* To facilitate some python practice, some of the data table formats used here will not necessarily be the ones used for your synapse tables / soma tables. 
* Try to reformat your data frames to get things to work!

In [1]:
import pandas as pd
import numpy as np
from annotationframeworkclient import FrameworkClient
import nglui
from matplotlib import cm
from nglui.statebuilder import *
from matplotlib import pyplot as plt
import pymaid
from cloudvolume import CloudVolume
from meshparty import trimesh_io, meshwork,mesh_filters,trimesh_vtk
from concurrent import futures
import json
import re
from pathlib import Path
def deserialize_pts(pt_string):
    vals = re.findall('\d+',pt_string)
    return([int(vals[0]),int(vals[1]),int(vals[2])])

## Establish FrameworkClient and CloudVolume objects to download tables.

In [5]:
with open(Path.home() / '.cloudvolume' / 'secrets'/'chunkedgraph-secret.json') as f:
        tokens = json.load(f)

with open(Path.home() / '.cloudvolume' / 'segmentations.json') as f:
        cv_paths = json.load(f)
        
#Client object

datastack_name = 'vnc_v0' # from https://api.zetta.ai/wclee/info/

client = FrameworkClient(
    datastack_name,
    server_address = "https://api.zetta.ai/wclee",
    auth_token = 'Q2h0UFNkM2ViazA5NTF2T2c1cTZiUXRyck5UTkN4dkk='
)

# CloudVolume object
cv = CloudVolume(cv_paths['Dynamic_V4']['url'],use_https=True)

In [4]:
with open(Path.home() / '.cloudvolume' / 'secrets'/'chunkedgraph-secret.json') as f:
        tokens = json.load(f)
        
tokens

{'token': 'Q2h0UFNkM2ViazA5NTF2T2c1cTZiUXRyck5UTkN4dkk='}

## First, we need to do two things:
1. Download our synapse table.
2. Download our mesh.

## Here we will use an already materialized synapse table and a 10B neuron. 
- The methods `.download_annotation_table` and `.generate_soma_table` should be familiar.
- If you do not have a soma table that includes your neuron, generate a DataFrame that includes a `pt_root_id` column containing the segment ID of your neuron and a `pt_position` column with the soma coordinates.
- To get a synapse table, adapt your code to generate a DataFrame from your synapse table with the columns `pre_pt` and `post_pt` which should be `[x,y,z]` coordinates for your synapses.

In [6]:
def download_annotation_table(client,table_name,ids=range(1000)):
    entries = client.annotation.get_annotation(table_name,ids)
    annotation_table = pd.DataFrame(entries)
    
    return(annotation_table)

def generate_soma_table(annotation_table,
                        segmentation_version='Dynamic_V1',
                        resolution=np.array([4.3,4.3,45]),
                        token=None):
    ''' Generate a soma table used for microns analysis. This is the workaround for a materialization engine
    Args:
        annotation_table: pd.DataFrame, output from download_cell_table. Retreived from the annotation engine.
        segmentation_version: str, Currently we have 4 for FANC. Two flat segmentations ("Flat_1" and "Flat_2") and two dynamic ("Dynamic_V1/V2"). 
                              This will only work if you have a segmentations.json in your cloudvolume folder. See examples for format.
        resolution: np.array, Resolution of the mip0 coordinates of the version (not necessarily the same as the segmentation layer resolution).
                              For all but the original FANC segmentation, this will be [4.3,4.3,45]
        token: str, currently, CloudVolume requires a workaround for passing google secret tokens. This won't work unless you edit your cloudvolume 
                              file to remove the check for hexidecimal formatting of tokens. Updates should be coming to fix this. 
        '''

    soma_table = pd.DataFrame(columns=['name','cell_type',
                                       'pt_position','pt_root_id',
                                       'soma_x_nm','soma_y_nm','soma_z_nm',
                                       'found'])
    with open(Path.home() / 'cloudvolume' / 'segmentations.json') as f:
            cloud_paths = json.load(f)
    if 'Dynamic' in segmentation_version:
        cv = CloudVolume(cloud_paths[segmentation_version]['url'],agglomerate=True,use_https=True,secrets=token)
    else:
        cv = CloudVolume(cloud_paths[segmentation_version]['url'],use_https=True,secrets=token)
        
    seg_ids = seg_from_pt(annotation_table.pt_position,cv)
    
    soma_table.name = annotation_table.tag
    soma_table.pt_position = annotation_table.pt_position
    soma_table.pt_root_id = seg_ids
    soma_table.soma_x_nm = np.array([i[0] for i in annotation_table.pt_position]) * resolution[0]
    soma_table.soma_y_nm = np.array([i[1] for i in annotation_table.pt_position]) * resolution[1]
    soma_table.soma_z_nm = np.array([i[2] for i in annotation_table.pt_position]) * resolution[2]
    
    return(soma_table)

def seg_from_pt(pts,vol,image_res=np.array([4.3,4.3,45]),max_workers=4):
    ''' Get segment ID at a point. Default volume is the static segmentation layer for now. 
    Args:
        pts (list): list of 3-element np.arrays of MIP0 coordinates
        vol_url (str): cloud volume url
    Returns:
        list, segment_ID at specified point '''
    
    
    seg_mip = vol.scale['resolution']
    res = seg_mip / image_res

    pts_scaled = [pt // res for pt in pts]
    results = []
    with futures.ThreadPoolExecutor(max_workers=max_workers) as ex:
        point_futures = [ex.submit(lambda pt,vol: vol[list(pt)][0][0][0][0], k,vol) for k in pts_scaled]
        
        for f in futures.as_completed(point_futures):
            results=[f.result() for f in point_futures]
       
    return results

In [7]:
# Get soma table:
table = download_annotation_table(client,'Hemilineages')
table_mat = generate_soma_table(table,token=tokens['dev'])
soma_df = table_mat.loc[table_mat.name == '10B']
# Load synapse table (UPDATE THIS FOR YOURSELF) It doesn't need to be from a CSV, it can be directly from the annotation engine. 
synapse_filename = '10Binputs.csv'
inputs = pd.read_csv(filename)
inputs.pre_pt = [deserialize_pts(i) for i in inputs.pre_pt]
inputs.post_pt = [deserialize_pts(i) for i in inputs.post_pt]


KeyError: 'dev'

## Supply a segment ID (your neuron of interest) and download the mesh.


In [ ]:
soma_df

In [9]:
neuron_id = 648518346508210135
mesh = cv.mesh.get(neuron_id,use_byte_offsets=True)[neuron_id]


## Format for meshwork
- Normally meshparty has its own i/o method, but it doesn't work for us, so we download a cloudvolume mesh and convert to a meshparty mesh.

In [10]:
mp_mesh = trimesh_io.Mesh(mesh.vertices,mesh.faces)

mp_mesh.merge_large_components(size_threshold=100,
                                    max_dist=1000,
                                    dist_step=500)
in_comp = mesh_filters.filter_largest_component(mp_mesh)
mesh_anchor = mp_mesh.apply_mask(in_comp)

neuron = meshwork.Meshwork(mesh_anchor, seg_id=neuron_id,voxel_resolution=[4.3,4.3,45])


5


/Users/brandon/anaconda3/envs/connectomics_analysis/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



Adding 24095 new edges.
TIME MERGING: 4.680s


In [23]:
si = meshwork.algorithms.strahler_order(neuron)


5

In [38]:
[si==5]

[392733.5113801724,
 392654.6729443061,
 392724.5073699883,
 392554.42290846753,
 392757.4267929252,
 392512.22605289635,
 392717.02779609646,
 392599.7346165685,
 392792.79527633905,
 392515.03762403806,
 392625.3203921011,
 392558.7633781725,
 392645.4818751898,
 392756.44708775944,
 392360.01634850615,
 392536.1043687574,
 392634.50776135304,
 392453.3263217424,
 392474.3045614579,
 392722.6512858661,
 392787.2874483083,
 391017.69789742253,
 392482.7949260961,
 392587.7452608525,
 392720.19407843036,
 391092.8504396367,
 392376.58034724754,
 392485.46371171507,
 390992.61050945707,
 391065.7472459075,
 391139.4852185087,
 392571.66982475953,
 392735.4612318577,
 392809.8538389789,
 392812.9883965142,
 391085.06645102164,
 392429.05763080285,
 392479.94473730755,
 392660.4617134249,
 392633.81693506736,
 392890.1436903196,
 391034.5931973283,
 392697.1236474747,
 392583.13317436347,
 392782.57564841135,
 392856.2717852421,
 391160.7299985519,
 392450.0697783095,
 392485.2451812679,


## Add annotations
- Meshwork has a bunch of cool methods for adding any type of annotation to our mesh object. While this may not be perfect for generating perfect skeletons as the whole process relies on meshparty, it is great for analysis. 
- Lets add our synapses (in the case of this example, input synapses) and a soma annotation.

In [ ]:
neuron.add_annotations('syn_in', inputs, point_column='post_pt')
neuron.add_annotations('soma_pt', soma_df.query('pt_root_id == @neuron_id').copy(), point_column='pt_position', anchored=False)


## Visualize our mesh and synapse annotations

In [ ]:
syn_actor = trimesh_vtk.point_cloud_actor(neuron.anno.syn_in.points, size=200, color=(0.2, 0.9, 0.9))
mesh_actor = trimesh_vtk.mesh_actor(neuron.mesh, opacity=.1, color=(0.7, 0.7, 0.7))


## Now we need to skeletonize our mesh. 
- Let's do that and visualize the result.

In [12]:
neuron.skeletonize_mesh(soma_pt=np.array([14669, 108852, 2728]),invalidation_distance=5000)
skel_actor = trimesh_vtk.skeleton_actor(neuron.skeleton, line_width=3, color=(0,0,0))
trimesh_vtk.render_actors([skel_actor, mesh_actor,syn_actor])

100%|██████████| 393333/393333 [00:15<00:00, 25514.73it/s] 


NameError: name 'mesh_actor' is not defined

## Density Analysis
- MeshWork has a few super useful algorithms including `linear_density` which will calculate the density of any annotation. 
- Let's calculate the input density and plot a mesh, then look at a histogram of densities. 

In [ ]:
rho = neuron.linear_density(neuron.anno.syn_in.mesh_index, 5000, normalize=True, exclude_root=True)
rho[np.isinf(rho)] = 0
ma = trimesh_vtk.mesh_actor(neuron.mesh, vertex_colors=(1000*rho-0.5), opacity=1)
trimesh_vtk.render_actors([ma])

In [ ]:
plt.hist(rho,range=[.0001,.0025])

## Now let's look at synapses on distinct arbors. 
- If a branch point exists between synapses, they are on distinct arbors so:
1. Find branch points.
2. Get synapse locations.
3. Find synapses distal to all branch points

In [ ]:
branch_points = neuron.branch_points
synapses = neuron.anno.syn_in.mesh_index
arbors = [neuron.downstream_of(i) for i in branch_points]
## ... you try the rest. 

## Alternatively, we can use branching order to prune our neuron and look at where synapses exist. 
* Branching order comes in a couple flavors:
  1. Count the branch points from the root to the node. (basic)
  2. Count the number of children of a node. (strahler) This one is more abstract, but arguably better: https://en.wikipedia.org/wiki/Strahler_number
  

In [17]:
def branch_mask(mw_nrn,
                method = 'basic',
                min_order = 1,
                max_order=None):
    ''' Apply a branching order mask to your neuron
    args:
    mw_nrn: meshwork neuron object
    method: str, basic or strahler, default is basic
    min_order: int, minimum branching order to include. Needs to be scaled depending on method. 
    max_order: int, maximum branching order to include. Needs to be scaled depending on method. 
    returns:
    mask: bool, mask for all vertices of your mesh. 
    '''
    if method == 'basic':
        si = meshwork.algorithms.branch_order(mw_nrn)
    elif method == 'strahler':
        si = meshwork.algorithms.strahler_order(mw_nrn)
    else:
        return('Wrong method')

    if max_order is None:
        max_order = np.max(si)
    mask = [min_order<=x<=max_order for x in si]
    return(mask)

In [19]:
neuron.reset_mask()
neuron.apply_mask(branch_mask(neuron,method='strahler',min_order=4))

In [21]:
ma = trimesh_vtk.mesh_actor(neuron.mesh, opacity=1)
trimesh_vtk.render_actors([ma])

setting up renderer
done setting up
actors added
camera set
render done
finalizing..


(vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer)0x7fb669817ad0

## How to apply this:
1. Do particular cell types synapse on different regions of the neuron?
   - Different branches, different branch orders (primary neurite vs twig, etc).
2. Do individual neurons cluster their synapses on particular arbors?
   - Either by density or by # of arbors they synapse onto, etc. 
3. Lots of other things... let me know if there are other tools that might help!